In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00


In [ ]:
!pip install pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [ ]:
!pip install torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
def join_keywords(keywords, randomize=True):
  N = len(keywords)
  if randomize:
      M = random.choice(range(N+1))
      keywords = keywords[:M]
      random.shuffle(keywords)

  return ','.join(keywords)

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}

device = torch.device("cuda")
tokenizer = AutoTokenizer.from_pretrained("NihalSrivastava/advertisement-description-generator")
model = AutoModelForCausalLM.from_pretrained("NihalSrivastava/advertisement-description-generator").to(device)

keywords = ['ring', 'diamond', 'luxury']
kw = join_keywords(keywords, randomize=False)
prompt = SPECIAL_TOKENS['bos_token'] + kw + SPECIAL_TOKENS['sep_token']
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)
model.eval();

Exception ignored in: <function _xla_gc_callback at 0x7fd5e57c1bd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [ ]:
# Beam-search text generation:
sample_outputs = model.generate(generated,
                                do_sample=True,
                                max_length=768,
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,
                                num_return_sequences=1
                                )
for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(','.join(keywords))
    print("{}: {}\n\n".format(i+1,  text[a:]))

1: The perfect addition to your collection of jewelry. Made with high-quality materials and available in a variety of colors and sizes, they're the perfect choice for any formal occasion.




In [ ]:
MAXLEN = 256
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated,
                                do_sample=True,
                                min_length=50,
                                max_length=MAXLEN,
                                top_k=30,
                                top_p=0.7,
                                temperature=0.9,
                                repetition_penalty=2.0,
                                num_return_sequences=10
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(','.join(keywords))
    print("{}: {}\n\n".format(i+1,  text[a:]))

1: Our jewelry has a stunning design and is perfect for any formal occasion. are the ultimate dress up with


2: Our jewelry has a sleek and balanced design that fits anywhere in your collection. they are the perfect addition to any outfit for everyone with our diamond earrings made from high-quality materials so you can get


3: Our jewelry collection has a large variety of colors and patterns available in an easy to pick up.


4: The perfect addition to your collection of jewelry. Made with high-quality materials and available in a variety colors for all styles at the office setting you can get ready today!


5: The jewelry ring has a sleek design and is perfect for any formal occasion. are made with high-quality materials so they can be used anywhere in your collection of colors or


6: The perfect way to start your day. Made with high-quality materials and available in a variety of styles they're the ultimate choice for any occasion on our website at home!


7: The perfect addition

# Fine tuning

In [2]:
# %%time
# %%capture
!pip install transformers

In [ ]:
!nvidia-smi

NotImplementedError: ignored

In [3]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler

from IPython.display import clear_output

print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.0.1+cu118


In [4]:
DEBUG           = False
USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'
MODEL           = 'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}
UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training
SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
MAXLEN          = 768  #{768, 1024, 1280, 1600}
TRAIN_SIZE      = 0.8
if USE_APEX:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 16
else:
    TRAIN_BATCHSIZE = 2
    BATCH_UPDATE    = 32
EPOCHS          = 2
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2
SEED            = 2020
EVALUATION_STRATEGY=2
SAVE_STRATEGY=2


In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [ ]:
def split_data(data, S=TRAIN_SIZE):
    # Shuffle ids
    ids = list(data.keys())
    random.shuffle(ids)
    print(len(ids))
    # Split into training and validation sets
    train_size = int(S * len(data))

    train_ids = ids[:train_size]
    val_ids = ids[train_size:]

    train_data = dict()
    for id in train_ids:
        train_data[id] = data[id]

    val_data = dict()
    for id in val_ids:
        val_data[id] = data[id]

    return train_data, val_data

In [ ]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")

df = pd.read_csv("drive/MyDrive/BPM/text generation/filtered_description_prompt.csv")
data = pd.DataFrame()
small = df.iloc[:10000]
data['keywords'] = [nlp(text).ents for text in small['prompt']]
print(data.head())
data['text'] = df['completion']
data['title'] = df['prompt']



                                            keywords
0  ((Computers, &, Accessories), (Cables, &, Acce...
1  ((Handhelds, &, Accessories), (Digital, Organi...
2  ((Audio, &, Video, Accessories), (Remote, Cont...
3  ((GPS, &, Navigation), (GPS), (Rand, McNally),...
4  ((Audio, &, Video, Accessories), (Headphones),...


In [ ]:
data


,keywords,text,title
0,"((Computers, &, Accessories), (Cables, &, Acce...",The Kelby Training DVD Mastering Blend Modes i...,Category is Electronics and Computers & Access...
1,"((Handhelds, &, Accessories), (Digital, Organi...",Digital Organizer and Messenger,Category is Electronics and Computers & Access...
2,"((Audio, &, Video, Accessories), (Remote, Cont...",The CLIKR-5 UR5U-8780L remote control is desig...,Category is Electronics and Accessories & Supp...
3,"((GPS, &, Navigation), (GPS), (Rand, McNally),...","Like its award-winning predecessor, the Intell...",Category is Electronics and GPS & Navigation a...
4,"((Audio, &, Video, Accessories), (Headphones),...",Use these high quality headphones for internet...,Category is Electronics and Accessories & Supp...
...,...,...,...
9995,"((Portable, Audio, &, Video), (Players, &, Acc...",Apple has revamped the now-classic iPod with s...,Category is Electronics and Portable Audio & V...
9996,"((Portable, Audio, &, Video), (Players, &, Acc...",Apple has revamped the now-classic iPod with s...,Category is Electronics and Portable Audio & V...
9997,"((Television, &, Video), (Home, Theater, Syste...",Kenwood gives you everything you need in your ...,Category is Electronics and Television & Video...
9998,"((Portable, Audio, &, Video), (Players, &, Acc...",The iPod is not just for Macs anymore. Windows...,Category is Electronics and Portable Audio & V...


In [ ]:
dt = pd.read_csv("drive/MyDrive/BPM/text generation/keywords_description.csv")
dt = dt.loc[:, ~dt.columns.str.contains('^Unnamed')]
dt.head()

,keywords,text,title
0,"(Computers & Accessories, Cables & Accessories...",The Kelby Training DVD Mastering Blend Modes i...,Category is Electronics and Computers & Access...
1,"(Handhelds & Accessories, Digital Organizer, M...",Digital Organizer and Messenger,Category is Electronics and Computers & Access...
2,"(Audio & Video Accessories, Remote Controls, T...",The CLIKR-5 UR5U-8780L remote control is desig...,Category is Electronics and Accessories & Supp...
3,"(GPS & Navigation, GPS, Rand McNally, 52888146...","Like its award-winning predecessor, the Intell...",Category is Electronics and GPS & Navigation a...
4,"(Audio & Video Accessories, Headphones, Polaro...",Use these high quality headphones for internet...,Category is Electronics and Accessories & Supp...


In [ ]:
dt.to_csv("drive/MyDrive/BPM/text generation/keywords_description.csv", index=False)

### Dataset and loaders

In [6]:
class myDataset(Dataset):

    def __init__(self, data, tokenizer, randomize=True):

        title, text, keywords = [], [], []

        for k, v in data.items():
            title.append(v[0])
            text.append(v[1])
            keywords.append(list(v[2]))

        self.randomize = randomize
        self.tokenizer = tokenizer
        self.title     = title
        self.text      = text
        self.keywords  = keywords

    #---------------------------------------------#

    @staticmethod
    def join_keywords(keywords, randomize=True):
        N = len(keywords)

        #random sampling and shuffle
        if randomize:
            M = random.choice(range(N+1))
            keywords = keywords[:M]
            random.shuffle(keywords)

        return ','.join(keywords)

    #---------------------------------------------#

    def __len__(self):
        return len(self.text)

    #---------------------------------------------#

    def __getitem__(self, i):

        # self.keywords[i] = list(self.keywords[i])
        keywords = self.keywords[i].copy()
        kw = self.join_keywords(keywords, self.randomize)

        input = SPECIAL_TOKENS['bos_token'] + self.title[i] + \
                SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token'] + \
                self.text[i] + SPECIAL_TOKENS['eos_token']

        encodings_dict = tokenizer(input,
                                   truncation=True,
                                   max_length=MAXLEN,
                                   padding="max_length")

        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']

        return {'label': torch.tensor(input_ids),
                'input_ids': torch.tensor(input_ids),
                'attention_mask': torch.tensor(attention_mask)}

In [ ]:

train_data, val_data = split_data(data)

train_dataset = myDataset(train_data, tokenizer)
val_dataset = myDataset(val_data, tokenizer, randomize=False)

f'There are {len(train_dataset) :,} samples for training, and {len(val_dataset) :,} samples for validation testing'

10000


'There are 8,000 samples for training, and 2,000 samples for validation testing'

In [ ]:
data = dict()

with open("drive/MyDrive/BPM/text generation/keywords_description.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)
    for i, row in enumerate(reader):
      data[i] = [row[2], row[1], row[0]]

In [ ]:
data = dict()

with open("drive/MyDrive/BPM/amazon/dataset.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)
    for i, row in enumerate(reader):
      data[i] = [row[0], row[1], row[2]]

In [ ]:
d = pd.read_csv("drive/MyDrive/BPM/amazon/dataset.csv")
d.head()

,title,text,keywords
0,Barnes & Noble Dessin Leather Cover for Nook C...,The Dessin Cover in Noir for Nook Tablet helps...,"['Noir', 'Nook Color', 'the Nook Tablet', 'Noi..."
1,8GB Owl Design USB Flash Drive,This beautiful 8G owl design flash drive is a ...,"['8', 'Panda']"
2,NB-6L Charger For Canon PowerShot D10 SD3500IS...,Charger Description: This high quality EconMax...,"['EconMax NB-6L / NB6L Battery Charger', 'only..."
3,Floor Stand Pop Up Portable Projector Screen 8...,"You are viewing a 80"" Portable Floor Stand Pro...","['80', 'Portable Floor Stand Projector Screen'..."
4,Professional Ultra SanDisk 16GB MicroSDHC Card...,Professional Ultra SanDisk 16GB MicroSDHC Noki...,"['16', 'GB', '520', 'UHS-1 Class 10 Certified'..."


In [ ]:
data[0]

['Category is Electronics and Computers & Accessories and Cables & Accessories and Monitor Accessories. Title is Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker.',
 'The Kelby Training DVD Mastering Blend Modes in Adobe Photoshop CS5 with Corey Barker is a useful tool for becoming familiar with the use of blend modes in Adobe Photoshop. For those who are serious about mastering all that Photoshop has to offer, mastering blend modes is just as important as mastering layers.In this DVD tutorial, seasoned expert Corey Barker explores the function of blend modes in a variety of scenarios such as image restoration, sharpening, adjustments, special effects and much more. Since every project scenario is different, Corey encourages you to experiment with these blend modes by giving you the skills and confidence you need.',
 '(Computers & Accessories, Cables & Accessories, Kelby Training, Modes, Corey Barker)']

### Loading Tokenizer, Config and Model

In [7]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL,
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else:
        config = AutoConfig.from_pretrained(MODEL,
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [8]:
%%time

tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer,
                  special_tokens=SPECIAL_TOKENS,
                #   load_model_path='pytorch_model.bin'
                 )

Special tokens added


CPU times: user 4.29 s, sys: 1.5 s, total: 5.79 s
Wall time: 12 s


In [9]:
# - Freeze selective layers:
# - Freeze all layers except last n:
for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True

for parameter in model.transformer.ln_f.parameters():
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():
    parameter.requires_grad = True

### Fine-tune GPT2 using Trainer

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 17.6 MB/s eta 0:00:00


In [ ]:
%%time

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/BPM/model_",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    gradient_accumulation_steps=BATCH_UPDATE,
    evaluation_strategy="steps",
    fp16=True,
    fp16_opt_level=APEX_OPT_LEVEL,
    warmup_steps=WARMUP_STEPS,
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
)

#---------------------------------------------------#
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

#---------------------------------------------------#
trainer.train()
trainer.save_model()

Step,Training Loss,Validation Loss


CPU times: user 4min 6s, sys: 2.63 s, total: 4min 8s
Wall time: 4min 25s


### Generate text

In [18]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer,
                  special_tokens=SPECIAL_TOKENS,
                  load_model_path='drive/MyDrive/BPM/model_/pytorch_model.bin')
print(model)

Special tokens added
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50262, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50262, bias=False)
)


In [19]:
title = "a suitable computer bag for office" #@param {type:"string"}
keywords = "computer bag, leather" #@param {type:""}
keys = []
for key in keywords.split(","):
  keys.append(key)

kw = myDataset.join_keywords(keys, randomize=False)

prompt = SPECIAL_TOKENS['bos_token'] + title + \
         SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token']

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [21]:
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated,
                                do_sample=True,
                                min_length=0,
                                max_length=75,
                                top_k=30,
                                top_p=0.6,
                                temperature=0.9,
                                repetition_penalty=2.0,
                                num_return_sequences=5
                                )
with open("drive/MyDrive/BPM/output_file/" +"_1".join(keywords)+".csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["text"])

    for i, sample_output in enumerate(sample_outputs):
      text = tokenizer.decode(sample_output, skip_special_tokens=True)
      a = len(title) + len(','.join(keys))
      print("{}: {}\n\n".format(i+1,  text[a:]))
      if text[a:] != "":
        writer.writerow([text[a:]])

1: The sleek and stylish Computer Bag is a high-quality custom design. It features an elegant mesh interior with  dividers to fit your laptop or desktop PC (2 x). The slim construction of the plastic


2: The Cyber-zone is a digital space  with the convenience of modern technology. The cyberZone provides protection from damage and spills while protecting your data or other products in this great home theater system(s). With its high performance


3: The Leather Jacket is designed to protect your laptop from scratches and other damage. It features a slim design with the  signature of an elegant pocket-sized size (2 inches). The sleekly padded pockets are filled upwith


4: The PC is the ideal size and compact design. The slim profile makes it  easy to fit your workstations or offices with this high-quality padded backpack of a single compartment that holds up most any desk space in


5: The sleek and stylish design of the Microsoft Office PC is designed to fit your desktop. The  ergonom

In [ ]:
# Beam-search text generation:
sample_outputs = model.generate(generated,
                                do_sample=True,
                                max_length=75,
                                num_beams=2,
                                repetition_penalty=2.0,
                                early_stopping=True,
                                num_return_sequences=1
                                )
with open("drive/MyDrive/BPM/output_file/beam" +"_".join(keywords)+".csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["text"])

    for i, sample_output in enumerate(sample_outputs):
      text = tokenizer.decode(sample_output, skip_special_tokens=True)
      a = len(title) + len(','.join(keys))
      print(text)
      print("{}: {}\n\n".format(i+1,  text[a:]))
      if text[a:] != "":
        writer.writerow([text[a:]])


a suitable computer bag for officecomputer bag, leatherThe Computer Bag is designed to hold your e-mail, fax, or other digital files. It includes a belt pocket and a  Accessory Kit. The Cyber-Bag makes it easy to take your e-
1: The Computer Bag is designed to hold your e-mail, fax, or other digital files. It includes a belt pocket and a  Accessory Kit. The Cyber-Bag makes it easy to take your e-




In [30]:
# Beam-search text generation:
sample_outputs = model.generate(generated,
                                do_sample=True,
                                max_length=75,
                                num_beams=4,
                                repetition_penalty=2.0,
                                early_stopping=True,
                                num_return_sequences=1
                                )
with open("drive/MyDrive/BPM/output_file/beam" +"_".join(keywords)+"_1.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["text"])

    for i, sample_output in enumerate(sample_outputs):
      text = tokenizer.decode(sample_output, skip_special_tokens=True)
      a = len(title) + len(','.join(keys))
      print(text)
      print("{}: {}\n\n".format(i+1,  text[a:]))
      if text[a:] != "":
        writer.writerow([text[a:]])


a suitable computer bag for officecomputer bag, leatherThe D-E400 is a high-performance, lightweight, compact, and stylish bag. The D-E400 is designed to meet your needs with its sleek design and easy-to-use
1: The D-E400 is a high-performance, lightweight, compact, and stylish bag. The D-E400 is designed to meet your needs with its sleek design and easy-to-use


